In [1]:
# import requests
# from pathlib import Path
# from zipfile import ZipFile
# import geopandas as gpd
# import numpy as np
# import plotly.graph_objects as go
# import json

# # get the shape file...
# url = "https://avoidatastr.blob.core.windows.net/avoindata/AvoinData/9_Kartat/PKS%20postinumeroalueet/Shp/PKS_postinumeroalueet_2021_shp.zip"

# fn = Path.cwd().joinpath(url.split("/")[-1])
# if not fn.exists():
#     r = requests.get(url, stream=True)
#     with open(fn, "wb") as f:
#         for chunk in r.raw.stream(1024, decode_content=False):
#             if chunk:
#                 f.write(chunk)

# zfile = ZipFile(fn)
# zfile.extractall()

# # open it...
# geodf = gpd.read_file(list(Path.cwd().glob("PKS*.shp"))[0])
# geodf["rand"] = np.random.randint(1, 100, len(geodf))

# # shape file is a different CRS,  change to lon/lat GPS co-ordinates
# geodf = geodf.to_crs("WGS84").set_index("Posno")


# fig = go.Figure(go.Choroplethmapbox(geojson=json.loads(geodf.to_json()), 
#                                     locations=geodf.index, z=geodf['rand'],
#                                     colorscale="Viridis", marker_line_width=.5))

# fig.update_layout(mapbox_style="open-street-map",
#                         height = 1000,
#                         autosize=True,
#                         margin={"r":0,"t":0,"l":0,"b":0},
#                         paper_bgcolor='#303030',
#                         plot_bgcolor='#303030',
#                         mapbox=dict(center=dict(lat=60.1699, lon=24.9384),zoom=9),
#                         )

In [2]:
import pandas as pd
import numpy as np
import requests
from tqdm.auto import tqdm, trange

In [3]:
df_raw = pd.read_csv("서울특별시_전동킥보드_견인_현황_20220731.csv", encoding="cp949", index_col=0)
df_raw

,신고일,구정보,주소,유형,조치일
번호,,,,,
1,2021-07-15,동작구,서울특별시 동작구 상도동 215-45,"점자블록, 엘리베이터 입구",2021-07-15
2,2021-07-15,동작구,서울특별시 동작구 상도동 172-28,보도 중앙,2021-07-15
3,2021-07-15,동작구,서울특별시 동작구 대방동 356-2,"버스정류장, 택시 승강장",2021-07-15
4,2021-07-15,동작구,서울특별시 동작구 대방동 354-1,"버스정류장, 택시 승강장",2021-07-15
5,2021-07-15,동작구,서울특별시 동작구 신대방동 460-30,보도와 차도가 구분된 도로의 차도,2021-07-15
...,...,...,...,...,...
60601,2022-07-29,동작구,서울특별시 동작구 사당동 708-1025,보도와 차도가 구분된 도로의 차도,2022-07-29
60602,2022-07-29,관악구,서울특별시 관악구 봉천동 1693-55,보도와 차도가 구분된 도로의 차도,2022-07-29
60603,2022-07-29,동작구,서울특별시 동작구 사당동 175-56,보도와 차도가 구분된 도로의 차도,2022-07-29


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60605 entries, 1 to 60605
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   신고일     60605 non-null  object
 1   구정보     60605 non-null  object
 2   주소      60605 non-null  object
 3   유형      60604 non-null  object
 4   조치일     60604 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [5]:
df_raw['날짜'] = pd.to_datetime(df_raw['신고일'])

In [6]:
df = df_raw[['날짜', '주소']]

In [7]:
# 날짜로부터 월, 주말 여부 컬럼 생성
df.insert(1, '월', df['날짜'].dt.month)
df.insert(2, "주말여부", df['날짜'].dt.dayofweek > 4)
df['주말여부'] = df['주말여부'].apply(lambda x: '주말' if x==True else '평일')

/var/folders/3m/knb96f9s4xg2t2gcrl96r6jh0000gn/T/ipykernel_15462/3973990059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주말여부'] = df['주말여부'].apply(lambda x: '주말' if x==True else '평일')


In [8]:
addrs = df['주소']
addrs

번호
1          서울특별시 동작구 상도동 215-45
2          서울특별시 동작구 상도동 172-28
3           서울특별시 동작구 대방동 356-2
4           서울특별시 동작구 대방동 354-1
5         서울특별시 동작구 신대방동 460-30
                  ...          
60601    서울특별시 동작구 사당동 708-1025
60602     서울특별시 관악구 봉천동 1693-55
60603      서울특별시 동작구 사당동 175-56
60604      서울특별시 동작구 사당동 175-26
60605      서울특별시 중랑구 면목동 197-10
Name: 주소, Length: 60605, dtype: object

In [9]:
locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK df5989bfdeb66e539382f8c9a0275d74"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)
    
    #d73f478209140e5db9caeb3ccca66181

  0%|          | 0/60605 [00:00<?, ?it/s]

In [10]:
len(locations)

60605

In [11]:
df.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            60596, 60597, 60598, 60599, 60600, 60601, 60602, 60603, 60604,
            60605],
           dtype='int64', name='번호', length=60605)

In [ ]:
for i in df.index:
    try:
        df.loc[i, 'x']= locations[i][0]['address']['x']
        df.loc[i, 'y']= locations[i][0]['address']['y']
        df.loc[i, '행정구']= locations[i][0]['address']['region_2depth_name']
        df.loc[i, '행정동']= locations[i][0]['address']['region_3depth_h_name']
    except:
        error_list.append(i)